Import some packages.

In [104]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

Read in the train dataset, generate some descriptive statistics for each variable.



In [47]:
url = "https://raw.githubusercontent.com/erik-sal/math154-kaggle/main/GiveMeSomeCredit/cs-training.csv?token=AMNU2Z7QXURLSSGSNZ4BOR27T4GCM"
data = pd.read_csv(url, index_col =0)

In [48]:
data.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


Look at where NA values are. We see that they live in columns 5 and 10. 

In [49]:
data.isna().sum()

SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

Impute medians of each column in place of any NA values.


In [57]:
data.fillna({"MonthlyIncome" : data.median()[5], "NumberOfDependents" : data.median()[10]}, inplace = True)

Now, we'll look at the distribution of our response variable, which is "SeriousDlqin2yrs"

In [76]:
data[data.columns[0]].value_counts()

0    139974
1     10026
Name: SeriousDlqin2yrs, dtype: int64

In [83]:
data[data.columns[0]].value_counts()[1]/(data[data.columns[0]].value_counts()[0] + data[data.columns[0]].value_counts()[1])

0.06684

Those who did experience a greater than 90 day past due delinquency of worse in the past two years form a small fraction of the observations. At the same time, based on the summary statistics generated earlier, there are clearly some egregious outliers in the values of observations (i.e. 98 times 30-59 past due but no worise in the past 2 years). So we'll just cut off the bottom 1 and top 1 percent of observations for each column to get rid of these, but to make sure that we're also not eliminating too many potentially-useful observations.

In [113]:
data_short = data

for i in range(1,10):
  column = data.columns[i]
  upperq = data[column].quantile(0.99)
  lowerq = data[column].quantile(0.01)
  data_short = data_short[(data_short[column] <= upperq) & (data_short[column] >= lowerq)]


In [115]:
data_short.reset_index(drop = True, inplace = True)
data_short

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
1,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
2,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
3,0,0.213179,74,0,0.375607,3500.0,3,0,1,0,1.0
4,0,0.754464,39,0,0.209940,3500.0,8,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
139098,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
139099,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
139100,0,0.246044,58,0,3870.000000,5400.0,18,0,1,0,0.0
139101,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [114]:
data_short.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000,139103.000000
mean,0.057576,0.307557,52.276011,0.211980,278.057479,5981.156496,8.246925,0.062752,0.958657,0.047778,0.740387
std,0.232941,0.341961,14.298638,0.584595,790.407232,3424.013607,4.628454,0.315466,0.926406,0.240392,1.105997
min,0.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029038,41.000000,0.000000,0.178632,3900.000000,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.145529,52.000000,0.000000,0.364085,5400.000000,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.527950,63.000000,0.000000,0.820738,7278.500000,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,1.092954,87.000000,4.000000,4979.000000,23000.000000,24.000000,3.000000,4.000000,2.000000,20.000000


We'll first try to do a basic kNN classifier on the dataset, with 5 nearest neighbors.

In [146]:
neigh = KNeighborsClassifier(n_neighbors = 1)
neigh.fit(data_short.iloc[:, 1:10], data_short.iloc[:,0])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [147]:
knnpred = neigh.predict(data_short.iloc[:, 1:10])

In [148]:
data_short["5NN"]  = knnpred

In [152]:
knncorrect = ((data_short["SeriousDlqin2yrs"] - data_short["5NN"]) == 0).sum()

In [153]:
knncorrect/data_short.shape[0]

0.9994895868529076

In [154]:
(data_short["SeriousDlqin2yrs"] == 0).sum()/data_short.shape[0]

0.9424239592244595

1NN does pretty great on the training set. 